<a href="https://colab.research.google.com/github/matteoalberti/Lectures_AdvCV_Experis2020/blob/main/day4/DL4CV_4_ObjectDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Object Detection





<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/08/real_time_object_detection.jpg" width="500">



   Email: m.alberti@deeplearningitalia.com

   Linkedin:
   [linkedin_matteo_alberti](www.linkedin.com/in/matteo-alberti-695570110)

# One step at a time : 

      Going Backwards!



## Which will be the core family behind deep learning models?

      We are working still with images

## How should we evaluate?

*Evaluation Metrics: mAP*

**mAP**, short for “mean average precision”.
- Range : from 0 to 100
- Higher value is better.

*What is?*

Combine all detections from all test images to draw a precision-recall curve (PR curve) for each class; The “average precision” (AP) is the area under the PR curve. (**Do you remember something?**)

Given that target objects are in different classes, we first compute AP separately for each class, and then average over classes.
A detection is a true positive if it has “intersection over union” (IoU) with a ground-truth box greater than some threshold (usually 0.5; if so, the metric is “mAP@0.5”)

### If you think carefully we have quite all blocks needed!

      Convolutions + new criterion

      What else?
      - New Data Structure
      - New Models

## Introduction CNN for Obj Detection

- <font color=BE3315>**R-CNN idea** </font> 
- <font color=C24024>**Variants like Fast, Faster and Mask** </font> 
- <font color=E15234>**SSD** </font> 

- <font color=E35F2A>**Exercises** </font>

## R-CNN

## Faster R-CNN

## Fastest R-CNN

## Mask R-CNN